In [1]:
from AOC_utils import get_day
import numpy as np

day = 12
input_data = get_day(day)

# print out first few lines to get a feel for the data
print(len(input_data))
input_data[:7]

Day 12 input already downloaded
140


['OOOBBBBXXXXXXXOOOOOOOAAAAAAAAAOOOOUUUUUUUUUUUUUUUUJJIFFFIIIIRRRRRRRRRRRRRRRRRRRRRNNNVVVVVVVVVVVPPPPPPPPPPPPPDDDDRRDDDDVVVVVVVVVVVGGVVGVVVVVV',
 'OOBBBBBXXXXXXOOOOOOOOOAAAAAAOOOOOOUUUUUUUUUUUUUUUUJJIFIIIIIIIRRRRAAARRRRRRRRRRRRRRVVVVVVVVVVVVVVPNNNNPPGPPPPDDDDDDDDDVVVVVVVVVVVVGGGVGGVVVVV',
 'OBBBBBBXXBXXOOOOOOOOOOAAAAAAOOOOOOUUUUUUUUUUUUUUUUJJIIIIIIIIRRRRAAAARRRRRRRRRRRRRRVVVVVVVVVVVVVFNNNNNPPGGPDDDDDDDDDDPPPVVPVVVVVVGGGGGGVVVVVV',
 'BBBBBBBBBBOOOOOOOOOOOOAAAAAAOOOOOOOOUUUUUUUUUUUUUUUIIIIIIIIRRRAAAAAAARRRRRRRGRRRRRRRVVVVVVVVVVVFNNNNNNPGGGGDBBDDDDDDDPPPPPPVVVGGGGGGGGGVVVVV',
 'ZZBBBBBBBBOOOOOOOOOOOOAAAAAAAOOOOOOOUUUUUUUUUUUUUIIIIIIIIIIIAAAAAAAAAARRARRRRRRBRRRVVVVVVVVVVVVVNNNNNNNNGGGBBBDDDDDDDPPPPPPPVVGGGGGGGGGGGVVV',
 'ZZBBBBBBBOOOOOODOOOOOOAAAAIIIROROOOOUUUUUUUUUUUUUUXIIIIIIIIIAAAAAAAAAAAAAAARRRRRRRRRVVVVVVVVVVVVNNNNNGGGGGGGBBWWWDDDDPPPPPPPGGGGGGGGGGGGGGVV',
 'ZZBBZBBBZOOOOOOOOOOOOOAAAAAIIRROOOOUUUPUUUUUUUUUUUIIIIIIIIIIAAAAAAAAAAADDAARRRRRRRRRRSVVVVVVVVNNNNNNNGGGGGGGGGWWDDDDDPYPPPPPGGGG

In [36]:
# part 1
example = '''
AAAA
BBCD
BBCC
EEEC
'''.split('\n')[1:-1]

In [34]:
# part 1
example = '''
OOOOO
OXOXO
OOOOO
OXOXO
OOOOO
'''.split('\n')[1:-1]

In [43]:
# part 1
example = '''
RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE
'''.split('\n')[1:-1]

In [103]:
def solve_day(data):
    part1 = 0
    part2 = 0

    data = np.array([[y for y in x] for x in data]) # 2d array
    # pad with *
    data = np.pad(data, 1, mode='constant', constant_values='*')
    # print(data)

    plot_kinds = np.unique(data)
    # remove star
    plot_kinds = plot_kinds[plot_kinds != '*']
    # print(plot_kinds)

    directions = np.array([[0, 1], [1, 0], [0, -1], [-1, 0]])

    all_regions = []
    region_types = []
    for kind in plot_kinds:
        where_data = np.zeros_like(data)
        where_data[data == kind] = 1

        where_kind = np.argwhere(data == kind)

        regions = []
        for plot in where_kind:
            if not regions:
                regions.append([plot])
            else:
                found_region = False
                for i in range(len(regions)):
                    for p in plot + directions:
                        if np.all(p == regions[i], axis=-1).any():
                            # print(kind, p, regions[i])
                            regions[i].append(plot)
                            found_region = True
                            break
                    if found_region:
                        break
                if not found_region:
                    # print(kind, "new region", plot)
                    regions.append([plot])

        # some regions may be connected, so we need to merge them
        for _ in range(4):
            merged_regions = []
            for region in regions:
                if not merged_regions:
                    merged_regions.append(region)
                else:
                    found_match = False
                    for i in range(len(merged_regions)):
                        for plot in region:
                            for p in plot + directions:
                                if np.all(p == merged_regions[i], axis=-1).any():
                                    merged_regions[i].extend(region)
                                    found_match = True
                                    break
                            if found_match:
                                break
                        if found_match:
                            break
                    if not found_match:
                        merged_regions.append(region)

            regions = merged_regions

        all_regions.extend(regions)
        region_types.extend([kind] * len(regions))
        # print(regions)

    areas = np.array([len(region) for region in all_regions])

    all_regions = [np.array(region) for region in all_regions]

    perimeters = []
    for region in all_regions:
        perimeter = 0
        for plot in region:
            for direction in directions:
                neighbor = plot + direction
                if data[neighbor[0], neighbor[1]] != data[plot[0], plot[1]]:
                    perimeter += 1
        perimeters.append(perimeter)
    perimeters = np.array(perimeters)

    sides = []
    for region in all_regions:
        side = 0
        new_data = np.zeros(data.shape)
        
        new_data[region[:,0], region[:,1]] = 1
        # new_data = new_data[~np.all(np.equal(new_data, 0), axis=1)]
        # new_data = new_data[:,~np.all(np.equal(new_data, 0), axis=0)]

        # pad with 0s
        new_data = np.pad(new_data, 1, mode='constant', constant_values=0)

        for i in range(1, len(new_data)-1):
            topinside = False
            botinside = False
            leftinside = False
            rightinside = False
            for j in range(1, len(new_data[i])-1):
                if new_data[i, j] == 1 and new_data[i-1, j] == 0:
                    if not topinside:
                        topinside = True
                else:
                    if topinside:
                        side += 1
                        topinside = False

                if new_data[i, j] == 1 and new_data[i+1, j] == 0:
                    if not botinside:
                        botinside = True
                else:
                    if botinside:
                        side += 1
                        botinside = False

                if new_data[j, i] == 1 and new_data[j, i-1] == 0:
                    if not leftinside:
                        leftinside = True
                else:
                    if leftinside:
                        side += 1
                        leftinside = False

                if new_data[j, i] == 1 and new_data[j, i+1] == 0:
                    if not rightinside:
                        rightinside = True
                else:
                    if rightinside:
                        side += 1
                        rightinside = False

        sides.append(side)

    # print(region_types)
    # print(areas)
    # print(perimeters)
    # print(sides)

    part1 = np.sum(areas * perimeters)
    part2 = np.sum(areas * sides)

    print("part 1:", part1)
    print("part 2:", part2)

solve_day(example)

part 1: 1930
part 2: 1206


In [104]:
solve_day(input_data)

part 1: 1473620
part 2: 902620
